In [1]:
import nest
import numpy as np
import pandas as pd

%matplotlib inline
import matplotlib.pyplot as plt
import time
        
import nest.voltage_trace
import nest.raster_plot
    
nest.set_verbosity("M_WARNING")

Network Architecture

In [ ]:
def CorticalThalamicNetwork():
    nest.ResetKernel()
    startbuild = time.time()
#
#   Parameter assumed by the network
    simtime = 1000.0
    pRate = 0.5
#   
#   Number of neuron populations
    ExciCrNeuron = 181 
    InhiCrNeuron = 201
    ExciThlNeuron = 325
    InhiThlNeuron = 201
    TotalNeurons = ExciCrNeuron + InhiCrNeuron + ExciThlNeuron + InhiThlNeuron
#
#   Neuron parameters -- first run
    vPeak = -40.4
    neuronParams = {"V_peak": vPeak}
#
    print("Building Cortical Thalamic Network")
#
#   Set defaults - AdEx com neuroParams and Poissonian spike generator with rate p_rate
    nest.SetDefaults("aeif_cond_alpha", neuronParams)
    nest.SetDefaults("poisson_generator", {"rate": pRate})                 
#
#   Creating populations
    NeuronsCxEx = nest.Create("aeif_cond_alpha", ExciCrNeuron)
    NeuronsCxIn = nest.Create("aeif_cond_alpha", InhiCrNeuron)
    NeuronsThEx = nest.Create("aeif_cond_alpha", ExciThlNeuron)
    NeuronsThIn = nest.Create("aeif_cond_alpha", InhiThlNeuron)
    NeuronsAll = NeuronsCxEx + NeuronsCxIn + NeuronsThEx + NeuronsThIn
#    
    noise = nest.Create("poisson_generator")
#
#   Creating voltmeters and recorders
    VmCxEx = nest.Create('voltmeter')
    VmCxIn = nest.Create("voltmeter")
    VmThEx = nest.Create("voltmeter")
    VmThIn = nest.Create("voltmeter")
    
    CrExspike = nest.Create("spike_recorder")
    CrInspike = nest.Create("spike_recorder")
    ThExspike = nest.Create("spike_recorder")
    ThInspike = nest.Create("spike_recorder")
    AllSpike = nest.Create("spike_recorder")
#
#   Configuration of the spike_recorders
    nest.Connect(VmCxEx, NeuronsCxEx)
    nest.Connect(VmCxIn, NeuronsCxIn)
    nest.Connect(VmThEx, NeuronsThEx)
    nest.Connect(VmThIn, NeuronsThIn)
    
    nest.Connect(NeuronsCxEx, CrExspike)
    nest.Connect(NeuronsCxIn, CrInspike)
    nest.Connect(NeuronsThEx, ThExspike)
    nest.Connect(NeuronsThIn, ThInspike)    
#
#   COULDN'T RUN THE VOLTMETERS 
#    nest.voltage_trace.from_device(VmCxEx)
#
    print("Implement the STDP")
#
#   Create the connection type between populations via STDP mechanism.
#   Excitatory connections from the Cortical population.
    CxExtoCxEx = {"synapse_model" : "stdp_synapse", "weight" : 1.0, "Wmax" : 150}
    CxExtoCxIn = {"synapse_model" : "stdp_synapse", "weight" : 60}
    CxExtoThEx = {"synapse_model" : "stdp_synapse", "weight" : 1.0, "Wmax" : 130}
#
#   Inhibitory connections from the Cortical population.
    CxIntoCxIn = {"synapse_model" : "stdp_synapse", "weight" : -1}
    CxIntoCxEx = {"synapse_model" : "stdp_synapse", "weight" : -4}
#
#   Excitatory connections from the Thalamical population.
    ThExtoCxEx = {"synapse_model" : "stdp_synapse", "weight" : 1.0, "Wmax" : 60}
    ThExtoThIn = {"synapse_model" : "stdp_synapse", "weight" : 10}
#
#   Inhibitory connections from the Thalamical population.
    ThIntoThIn = {"synapse_model" : "stdp_synapse", "weight" : -1}
    ThIntoThEx = {"synapse_model" : "stdp_synapse", "weight" : -10}
#
    print(ThIntoThIn)
    print(ThIntoThEx)
#   Connecting populations
    nest.Connect(NeuronsCxEx, 
                 NeuronsCxEx,
                 syn_spec=CxExtoCxEx)
#
    nest.Connect(NeuronsCxEx, 
                 NeuronsCxIn,
                 syn_spec=CxExtoCxIn)
#
    nest.Connect(NeuronsCxEx, 
                 NeuronsThEx,
                 syn_spec=CxExtoThEx)
#
    nest.Connect(NeuronsThEx,
                 NeuronsCxEx,
                 syn_spec=ThExtoCxEx)
#
    nest.Connect(NeuronsThEx,
                 NeuronsThIn,
                 syn_spec=ThExtoThIn)
#
    nest.Connect(NeuronsCxIn,
                 NeuronsCxIn,
                 syn_spec=CxIntoCxIn)
#
    nest.Connect(NeuronsCxIn,
                 NeuronsCxEx,
                 syn_spec=CxIntoCxEx)
#
    nest.Connect(NeuronsThIn,
                 NeuronsThIn,
                 syn_spec=ThIntoThIn)
#
    nest.Connect(NeuronsThIn,
                 NeuronsThEx,
                 syn_spec=ThIntoThEx)
#
#   Activantig the Possoinian spike train over 20 cortical excitatory neurons
    CxExNoise = nest.Create('poisson_generator', {'rate': 2000.0})
    CxInNoise = nest.Create('poisson_generator', {'rate': 10000.0})
    nest.Connect(CxExNoise, NeuronsCxEx[20:40], syn_spec={'weight':15})
    nest.Connect(CxInNoise, NeuronsCxIn, syn_spec={'weight':5})
#
    nest.Simulate(simtime)
#
    N_rec = 100.0
    rateEx= CxExNoise.n_events / simtime * 1000 / N_rec
    print(f"Excitatory rate: {rateEx:.2f} [1/s]")

    rateIn= CxInNoise.n_events / simtime * 1000 / N_rec
    print(f"Inhibitory rate: {rateIn:.2f} [1/s]")
#
    nest.raster_plot.from_device(CxExNoise, hist=True)
    nest.raster_plot.from_device(CxInNoise, hist=True)
#

In [ ]:
CorticalThalamicNetwork()

In [ ]:
#nest.GetDefaults('voltmeter')